In [ ]:
!pip install pretrainedmodels > /dev/null 2>&1

import numpy as np_custom  
import pandas as pd_custom  
import os_custom  
import cv2_custom  
import matplotlib.pyplot as plt_custom  
%matplotlib inline

from sklearn.model_selection import train_test_split_custom  
from sklearn.metrics import roc_auc_score_custom  
import torch_custom  
from torch.utils.data import TensorDataset_custom, DataLoader_custom, Dataset_custom  
import torch.nn as nn_custom  
import torch.nn.functional as F_custom 
import torchvision_custom  
import torchvision.transforms as transforms_custom 
import torch.optim as optim_custom  
from torch.optim import lr_scheduler_custom  
import time_custom  
import tqdm_custom  
from PIL import Image_custom 

train_on_gpu_custom = True  

from torch.utils.data.sampler import SubsetRandomSampler_custom  
from torch.optim.lr_scheduler import StepLR_custom, ReduceLROnPlateau_custom, CosineAnnealingLR_custom  
import cv2_custom  


In [ ]:
!pip install albumentations > /dev/null 2>&1

In [ ]:
!pip install pretrainedmodels > /dev/null 2>&1

In [ ]:
import albumentations_custom  
from albumentations_custom import torch as AT_custom  
import pretrainedmodels_custom  


In [ ]:
labels_custom = pd_custom.read_csv('../input/train_labels.csv')  


In [ ]:
print(f'{len(os_custom.listdir("../input/train"))} pictures in train.')
print(f'{len(os_custom.listdir("../input/test"))} pictures in test.')


In [ ]:
fig_custom = plt_custom.figure(figsize=(25, 4))
train_imgs_custom = os_custom.listdir("../input/train")
for idx_custom, img_custom in enumerate(np_custom.random.choice(train_imgs_custom, 20)):
    ax_custom = fig_custom.add_subplot(2, 20//2, idx_custom+1, xticks=[], yticks=[])
    im_custom = Image_custom.open("../input/train/" + img_custom)
    plt_custom.imshow(im_custom)
    lab_custom = labels_custom.loc[labels_custom['id'] == img_custom.split('.')[0], 'label'].values[0]
    ax_custom.set_title(f'Label: {lab_custom}')


In [ ]:
labels_custom.label.value_counts()

In [ ]:
data_transforms_custom = transforms_custom.Compose([
    transforms_custom.CenterCrop(32),
    transforms_custom.RandomHorizontalFlip(),
    transforms_custom.RandomVerticalFlip(),
    transforms_custom.ToTensor(),
    transforms_custom.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

data_transforms_test_custom = transforms_custom.Compose([
    transforms_custom.CenterCrop(32),
    transforms_custom.ToTensor(),
    transforms_custom.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
tr_custom, val_custom = train_test_split_custom(labels_custom.label, stratify=labels_custom.label, test_size=0.1)
img_class_dict_custom = {k: v for k, v in zip(labels_custom.id, labels_custom.label)}



In [ ]:
class CancerDataset_custom(Dataset_custom):
    def __init__(self, datafolder_custom, datatype_custom='train', transform_custom=transforms_custom.Compose([transforms_custom.CenterCrop(32), transforms_custom.ToTensor()]), labels_dict_custom={}):
        self.datafolder_custom = datafolder_custom
        self.datatype_custom = datatype_custom
        self.image_files_list_custom = [s for s in os_custom.listdir(datafolder_custom)]
        self.transform_custom = transform_custom
        self.labels_dict_custom = labels_dict_custom
        if self.datatype_custom == 'train':
            self.labels_custom = [labels_dict_custom[i.split('.')[0]] for i in self.image_files_list_custom]
        else:
            self.labels_custom = [0 for _ in range(len(self.image_files_list_custom))]

    def __len__(self):
        return len(self.image_files_list_custom)

    def __getitem__(self, idx_custom):
        img_name_custom = os_custom.path.join(self.datafolder_custom, self.image_files_list_custom[idx_custom])
        image_custom = Image_custom.open(img_name_custom)
        image_custom = self.transform_custom(image_custom)
        img_name_short_custom = self.image_files_list_custom[idx_custom].split('.')[0]

        if self.datatype_custom == 'train':
            label_custom = self.labels_dict_custom[img_name_short_custom]
        else:
            label_custom = 0
        return image_custom, label_custom


In [ ]:
%%time
dataset_custom = CancerDataset_custom(datafolder_custom='../input/train/', datatype_custom='train', transform_custom=data_transforms_custom, labels_dict_custom=img_class_dict_custom)
test_set_custom = CancerDataset_custom(datafolder_custom='../input/test/', datatype_custom='test', transform_custom=data_transforms_test_custom)


In [ ]:
train_sampler_custom = SubsetRandomSampler_custom(list(tr_custom.index))
valid_sampler_custom = SubsetRandomSampler_custom(list(val_custom.index))
batch_size_custom = 512
num_workers_custom = 0
train_loader_custom = torch_custom.utils.data.DataLoader(dataset_custom, batch_size=batch_size_custom, sampler=train_sampler_custom, num_workers=num_workers_custom)
valid_loader_custom = torch_custom.utils.data.DataLoader(dataset_custom, batch_size=batch_size_custom, sampler=valid_sampler_custom, num_workers=num_workers_custom)
test_loader_custom = torch_custom.utils.data.DataLoader(test_set_custom, batch_size=batch_size_custom, num_workers=num_workers_custom)


In [ ]:
class Net_custom(nn_custom.Module):
    def __init__(self):
        super(Net_custom, self).__init__()
        self.conv1 = nn_custom.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn_custom.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn_custom.Conv2d(32, 64, 3, padding=1)
        self.pool = nn_custom.MaxPool2d(2, 2)
        self.fc1 = nn_custom.Linear(64 * 4 * 4, 512)
        self.fc2 = nn_custom.Linear(512, 256)
        self.fc3 = nn_custom.Linear(256, 2)
        self.dropout = nn_custom.Dropout(0.2)
        self.sig = nn_custom.Sigmoid()

    def forward(self, x_custom):
        x_custom = self.pool(F_custom.relu(self.conv1(x_custom)))
        x_custom = self.pool(F_custom.relu(self.conv2(x_custom)))
        x_custom = self.pool(F_custom.relu(self.conv3(x_custom)))
        x_custom = x_custom.view(-1, 64 * 4 * 4)
        # x_custom = self.dropout(x_custom)
        x_custom = F_custom.relu(self.fc1(x_custom))
        x_custom = self.dropout(x_custom)
        x_custom = F_custom.relu(self.fc2(x_custom))
        x_custom = self.sig(self.fc3(x_custom))
        return x_custom


In [ ]:
criterion_custom = nn_custom.BCELoss()


In [ ]:
class CancerDataset_custom(Dataset_custom):
    def __init__(self, datafolder_custom, datatype_custom='train', transform_custom=transforms_custom.Compose([transforms_custom.CenterCrop(32), transforms_custom.ToTensor()]), labels_dict_custom={}):
        self.datafolder_custom = datafolder_custom
        self.datatype_custom = datatype_custom
        self.image_files_list_custom = [s for s in os_custom.listdir(datafolder_custom)]
        self.transform_custom = transform_custom
        self.labels_dict_custom = labels_dict_custom
        if self.datatype_custom == 'train':
            self.labels_custom = [labels_dict_custom[i.split('.')[0]] for i in self.image_files_list_custom]
        else:
            self.labels_custom = [0 for _ in range(len(self.image_files_list_custom))]

    def __len__(self):
        return len(self.image_files_list_custom)

    def __getitem__(self, idx_custom):
        img_name_custom = os_custom.path.join(self.datafolder_custom, self.image_files_list_custom[idx_custom])
        img_custom = cv2_custom.imread(img_name_custom)
        img_custom = cv2_custom.cvtColor(img_custom, cv2_custom.COLOR_BGR2RGB)
        image_custom = self.transform_custom(image_custom=img_custom)
        image_custom = image_custom['image']

        img_name_short_custom = self.image_files_list_custom[idx_custom].split('.')[0]

        if self.datatype_custom == 'train':
            label_custom = self.labels_dict_custom[img_name_short_custom]
        else:
            label_custom = 0
        return image_custom, label_custom


In [ ]:
data_transforms_custom = albumentations_custom.Compose([
    albumentations_custom.Resize(224, 224),
    albumentations_custom.HorizontalFlip(),
    albumentations_custom.RandomBrightness(),
    albumentations_custom.ShiftScaleRotate(rotate_limit=15, scale_limit=0.10),
    albumentations_custom.JpegCompression(80),
    albumentations_custom.HueSaturationValue(),
    albumentations_custom.Normalize(),
    AT_custom.ToTensor()
])

data_transforms_test_custom = albumentations_custom.Compose([
    albumentations_custom.Resize(224, 224),
    albumentations_custom.Normalize(),
    AT_custom.ToTensor()
])

dataset_custom = CancerDataset_custom(datafolder_custom='../input/train/', datatype_custom='train', transform_custom=data_transforms_custom, labels_dict_custom=img_class_dict_custom)
test_set_custom = CancerDataset_custom(datafolder_custom='../input/test/', datatype_custom='test', transform_custom=data_transforms_test_custom)
train_sampler_custom = SubsetRandomSampler_custom(list(tr_custom.index))
valid_sampler_custom = SubsetRandomSampler_custom(list(val_custom.index))
batch_size_custom = 32
num_workers_custom = 0
train_loader_custom = torch_custom.utils.data.DataLoader(dataset_custom, batch_size=batch_size_custom, sampler=train_sampler_custom, num_workers=num_workers_custom)
valid_loader_custom = torch_custom.utils.data.DataLoader(dataset_custom, batch_size=batch_size_custom, sampler=valid_sampler_custom, num_workers=num_workers_custom)
test_loader_custom = torch_custom.utils.data.DataLoader(test_set_custom, batch_size=batch_size_custom, num_workers=num_workers_custom)


In [ ]:
model_conv_custom = torchvision_custom.models.resnet50(pretrained=True)
for i_custom, param_custom in model_conv_custom.named_parameters():
    param_custom.requires_grad = False


In [ ]:
num_ftrs_custom = model_conv_custom.fc.in_features
model_conv_custom.fc = nn_custom.Linear(2048, 2)


In [ ]:
model_conv_custom.cuda()
criterion_custom = nn_custom.BCEWithLogitsLoss()

optimizer_custom = optim_custom.SGD(model_conv_custom.fc.parameters(), lr=0.005, momentum=0.99)
scheduler_custom = lr_scheduler_custom.StepLR(optimizer_custom, step_size=3, gamma=0.1)


In [ ]:
valid_loss_min_custom = np_custom.Inf
patience_custom = 5
p_custom = 0
stop_custom = False

n_epochs_custom = 7
for epoch_custom in range(1, n_epochs_custom+1):
    print(time_custom.ctime(), 'Epoch:', epoch_custom)

    train_loss_custom = []
    train_auc_custom = []

    for batch_i_custom, (data_custom, target_custom) in enumerate(train_loader_custom):

        data_custom, target_custom = data_custom.cuda(), target_custom.cuda()

        optimizer_custom.zero_grad()
        output_custom = model_conv_custom(data_custom)
        loss_custom = criterion_custom(output_custom[:,1], target_custom.float())
        train_loss_custom.append(loss_custom.item())
        
        a_custom = target_custom.data.cpu().numpy()
        b_custom = output_custom[:,-1].detach().cpu().numpy()

        loss_custom.backward()
        optimizer_custom.step()
    
    model_conv_custom.eval()
    val_loss_custom = []
    val_auc_custom = []
    for batch_i_custom, (data_custom, target_custom) in enumerate(valid_loader_custom):
        data_custom, target_custom = data_custom.cuda(), target_custom.cuda()
        output_custom = model_conv_custom(data_custom)

        loss_custom = criterion_custom(output_custom[:,1], target_custom.float())

        val_loss_custom.append(loss_custom.item()) 
        a_custom = target_custom.data.cpu().numpy()
        b_custom = output_custom[:,-1].detach().cpu().numpy()

    print(f'Epoch {epoch_custom}, train loss: {np_custom.mean(train_loss_custom):.4f}, valid loss: {np_custom.mean(val_loss_custom):.4f}.')
    
    valid_loss_custom = np_custom.mean(val_loss_custom)
    scheduler_custom.step(valid_loss_custom)
    if valid_loss_custom <= valid_loss_min_custom:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min_custom,
        valid_loss_custom))
        torch_custom.save(model_conv_custom.state_dict(), 'model.pt')
        valid_loss_min_custom = valid_loss_custom
        p_custom = 0

    if valid_loss_custom > valid_loss_min_custom:
        p_custom += 1
        print(f'{p_custom} epochs of increasing val loss')
        if p_custom > patience_custom:
            print('Stopping training')
            stop_custom = True
            break        
            
    if stop_custom:
        break


In [ ]:
model_conv_custom.eval()
preds_custom = []
for batch_i_custom, (data_custom, target_custom) in enumerate(test_loader_custom):
    data_custom, target_custom = data_custom.cuda(), target_custom.cuda()
    output_custom = model_conv_custom(data_custom)

    pr_custom = output_custom[:,1].detach().cpu().numpy()
    for i_custom in pr_custom:
        preds_custom.append(i_custom)


In [ ]:
test_preds_custom = pd_custom.DataFrame({'imgs': test_set_custom.image_files_list_custom, 'preds': preds_custom})


In [ ]:
test_preds_custom['imgs'] = test_preds_custom['imgs'].apply(lambda x_custom: x_custom.split('.')[0])


In [ ]:
sub_custom = pd_custom.read_csv('../input/sample_submission.csv')


In [ ]:
sub_custom = pd_custom.merge(sub_custom, test_preds_custom, left_on='id', right_on='imgs')
sub_custom = sub_custom[['id', 'preds']]
sub_custom.columns = ['id', 'label']
sub_custom.head()


In [ ]:
sub_custom.to_csv('sub.csv', index=False)
